# Tree 계열 모델에 input 값으로 들어갈 데이터 생성해주기

- RF, LGBM Input data 만들기
    - 각 행에 앞 20개 행 데이터 컬럼으로 생성(20 sequence)
- 생성일자: 2024.04.04
- 파이썬 버전: 3.8.18

In [2]:
import pandas as pd
import numpy as np

In [3]:
file_path = '../../data/' 
df = pd.read_csv(file_path + 'bitcoin_data_num_rows_gt_5.csv')
df = df.sort_values(by='window_start', ascending=True) # 시간순 정렬
print(df.shape)
df.head()

(149559, 85)


,window_start,window_end,num_rows,lowest_return,highest_return,high_low_gap,trade_vol,volume_power,beginning_price,ending_price,...,ob_end_bs_14,ob_end_bias_0,ob_end_bias_1,ob_end_bias_4,ob_end_bidask_spread,ob_end_liq_0,ob_end_liq_1,ob_end_liq_4,highest_possible_return,del_idx
0,2022-12-16 21:05:30,2022-12-16 21:06:00,14,0.000000,0.000089,0.000089,1.468656,0.747351,22568000.0,22570000.0,...,1.467714,5.470422,10.649683,3.235541,2.0,0.001693,0.002198,0.002412,1.000000,0
1,2022-12-16 21:06:00,2022-12-16 21:06:30,10,0.000000,0.000089,0.000089,0.567585,0.027857,22568000.0,22570000.0,...,0.143039,4.224361,14.918538,3.856600,2.0,0.000531,0.001064,0.001471,1.000000,0
2,2022-12-16 21:06:30,2022-12-16 21:07:00,24,-0.000576,0.000044,0.000620,1.677093,0.146635,22570000.0,22570000.0,...,0.271898,17.677511,9.697905,1.106227,14.0,0.000449,0.000536,0.001821,0.999778,0
3,2022-12-16 21:07:00,2022-12-16 21:07:30,22,-0.000044,0.000443,0.000488,2.439677,0.751995,22557000.0,22567000.0,...,0.640898,95.630870,3.371113,1.367349,2.0,0.000416,0.000480,0.001422,0.999911,0
4,2022-12-16 21:07:30,2022-12-16 21:08:00,24,-0.000443,0.000000,0.000443,2.345821,-0.915608,22565000.0,22555000.0,...,0.081040,0.114815,0.828364,0.068175,10.0,0.000311,0.000560,0.003454,0.999911,0


In [4]:
# 종속변수 리스트
target_var_lst = ['returns', 'returns_next10m', 'realized_vol_next10m']
target_var = 'returns_next10m' # 종속변수

# 시퀀스 생성 전 필요없는 컬럼 삭제
df.drop(columns=['window_start', 'window_end','num_rows', 'time_id'], inplace=True) # 시간 관련 변수

# target을 제외한 나머지 종속변수 삭제
cols_to_drop = [var for var in target_var_lst if var != target_var]
df.drop(columns=cols_to_drop, inplace=True) # 종속변수
df.drop(columns='del_idx', axis=1, inplace=True) # 필요 없는 변수


# 종속변수를 데이터 프레임 맨 뒤로 옮기기
cols = df.columns.tolist()
cols = [col for col in cols if col != 'returns_next10m'] + ['returns_next10m'] # 종속변수 맨 뒤로
df = df[cols]
df.columns

Index(['lowest_return', 'highest_return', 'high_low_gap', 'trade_vol',
       'volume_power', 'beginning_price', 'ending_price', 'lowest_price',
       'highest_price', 'ob_end_ap_0', 'ob_end_as_0', 'ob_end_bp_0',
       'ob_end_bs_0', 'ob_end_ap_1', 'ob_end_as_1', 'ob_end_bp_1',
       'ob_end_bs_1', 'ob_end_ap_2', 'ob_end_as_2', 'ob_end_bp_2',
       'ob_end_bs_2', 'ob_end_ap_3', 'ob_end_as_3', 'ob_end_bp_3',
       'ob_end_bs_3', 'ob_end_ap_4', 'ob_end_as_4', 'ob_end_bp_4',
       'ob_end_bs_4', 'ob_end_ap_5', 'ob_end_as_5', 'ob_end_bp_5',
       'ob_end_bs_5', 'ob_end_ap_6', 'ob_end_as_6', 'ob_end_bp_6',
       'ob_end_bs_6', 'ob_end_ap_7', 'ob_end_as_7', 'ob_end_bp_7',
       'ob_end_bs_7', 'ob_end_ap_8', 'ob_end_as_8', 'ob_end_bp_8',
       'ob_end_bs_8', 'ob_end_ap_9', 'ob_end_as_9', 'ob_end_bp_9',
       'ob_end_bs_9', 'ob_end_ap_10', 'ob_end_as_10', 'ob_end_bp_10',
       'ob_end_bs_10', 'ob_end_ap_11', 'ob_end_as_11', 'ob_end_bp_11',
       'ob_end_bs_11', 'ob_end_ap_12', 'ob

In [5]:
df

,lowest_return,highest_return,high_low_gap,trade_vol,volume_power,beginning_price,ending_price,lowest_price,highest_price,ob_end_ap_0,...,ob_end_bs_14,ob_end_bias_0,ob_end_bias_1,ob_end_bias_4,ob_end_bidask_spread,ob_end_liq_0,ob_end_liq_1,ob_end_liq_4,highest_possible_return,returns_next10m
0,0.000000,0.000089,0.000089,1.468656,0.747351,22568000.0,22570000.0,22568000.0,22570000.0,22570000.0,...,1.467714,5.470422,10.649683,3.235541,2.0,0.001693,0.002198,0.002412,1.000000,-0.000532
1,0.000000,0.000089,0.000089,0.567585,0.027857,22568000.0,22570000.0,22568000.0,22570000.0,22570000.0,...,0.143039,4.224361,14.918538,3.856600,2.0,0.000531,0.001064,0.001471,1.000000,-0.000310
2,-0.000576,0.000044,0.000620,1.677093,0.146635,22570000.0,22570000.0,22557000.0,22571000.0,22571000.0,...,0.271898,17.677511,9.697905,1.106227,14.0,0.000449,0.000536,0.001821,0.999778,-0.000133
3,-0.000044,0.000443,0.000488,2.439677,0.751995,22557000.0,22567000.0,22556000.0,22567000.0,22567000.0,...,0.640898,95.630870,3.371113,1.367349,2.0,0.000416,0.000480,0.001422,0.999911,-0.000532
4,-0.000443,0.000000,0.000443,2.345821,-0.915608,22565000.0,22555000.0,22555000.0,22565000.0,22565000.0,...,0.081040,0.114815,0.828364,0.068175,10.0,0.000311,0.000560,0.003454,0.999911,0.000399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149554,0.000000,0.000033,0.000033,0.153247,0.802207,30507000.0,30508000.0,30507000.0,30508000.0,30509000.0,...,0.050066,0.000366,0.555042,0.257513,1.0,0.000637,0.000937,0.000995,1.000295,0.000295
149555,-0.000033,0.000000,0.000033,0.879080,-0.505112,30509000.0,30509000.0,30508000.0,30509000.0,30509000.0,...,0.076681,2.503651,5.778803,4.708884,1.0,0.002924,0.003727,0.003737,1.000361,0.000361
149556,-0.000033,0.000000,0.000033,0.412820,0.509523,30509000.0,30508000.0,30508000.0,30509000.0,30509000.0,...,2.110355,0.982967,2.550520,2.278613,1.0,0.002965,0.003754,0.003763,1.000393,0.000426
149557,0.000000,0.000033,0.000033,2.141030,0.785751,30509000.0,30510000.0,30509000.0,30510000.0,30513000.0,...,1.824012,0.014928,0.000845,1.153614,4.0,0.000006,0.000094,0.000185,1.000295,0.000295


In [6]:
def create_lagged_features(df, n_lags):
    """
    df: 원본 데이터프레임
    n_lags: 이전 데이터 포인트의 수 (예: 20이면, 이전 20개의 데이터 포인트에 대한 새 컬럼을 생성)
    """
    # 종속 변수 제외
    features = df.drop(columns=['returns_next10m'])
    lagged_df = df.copy()  # 원본 데이터프레임 복사
    
    # 각 컬럼에 대해 지정된 lag만큼의 새 컬럼 생성
    for col in features.columns:
        for lag in range(1, n_lags + 1):
            lagged_df[f'{col}_bf_{lag}'] = features[col].shift(lag)
    
    return lagged_df

In [7]:
new_df = create_lagged_features(df, 20)

/var/folders/yd/1_xwcyjj6z58p2vptxk1dwvm0000gn/T/ipykernel_47797/2500803085.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  lagged_df[f'{col}_bf_{lag}'] = features[col].shift(lag)
/var/folders/yd/1_xwcyjj6z58p2vptxk1dwvm0000gn/T/ipykernel_47797/2500803085.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  lagged_df[f'{col}_bf_{lag}'] = features[col].shift(lag)
/var/folders/yd/1_xwcyjj6z58p2vptxk1dwvm0000gn/T/ipykernel_47797/2500803085.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result o

In [8]:
# 데이터 내보내기
new_df.to_csv(f"{file_path}bitcoin_data_num_rows_gt_5_for_treemodel.csv", index=False)